In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, get_peft_model


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)


model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
prompt_template = "### Question:\n{question}\n\n### Answer:\n"


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_base = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)
model_base.resize_token_embeddings(len(tokenizer))

model_base.config.pad_token_id = tokenizer.pad_token_id

In [3]:
def generate_response(model_instance, question):
    prompt = prompt_template.format(question=question)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(model_instance.device)
    output = model_instance.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):]
    answer = generated_text.split("### Question:")[0]
    return answer.strip()

In [4]:

df = pd.read_csv("/content/medquad.csv", on_bad_lines='skip', engine='python').head(1000)
dataset = Dataset.from_pandas(df[["question", "answer"]])


def tokenize(example):
    prompt = prompt_template.format(question=example["question"])
    full_text = prompt + example["answer"]
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, remove_columns=["question", "answer"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

sample_question = df["question"].iloc[0]
print("\n--- BEFORE FINE-TUNING---")
print("Question:", sample_question)
print("Answer:", generate_response(model_base, sample_question))


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


--- BEFORE FINE-TUNING (Base Model) ---
Question: What is (are) Glaucoma ?
Answer: Glaucoma is a disease of the eye that causes vision loss, and it can lead to blindness if not treated.


In [ ]:

model_ft = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)
model_ft.resize_token_embeddings(len(tokenizer))
model_ft.config.pad_token_id = tokenizer.pad_token_id

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model_ft = get_peft_model(model_ft, lora_config)
model_ft.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./tinyllama_lora_finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=850,
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

print("\n--- AFTER FINE-TUNING---")
print("Question:", sample_question)
print("Answer:", generate_response(model_ft, sample_question))

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


Step,Training Loss
10,1.910300
20,1.669600
30,1.500700
40,1.380300
50,1.320800
60,1.312500
70,1.321200
80,1.295300
90,1.290200
100,1.285000



--- AFTER FINE-TUNING (LoRA Fine-Tuned Model) ---
Question: What is (are) Glaucoma ?
Answer: Glaucoma is an eye disease in which the pressure in the eye becomes too high. The eye's natural drainage system, the optic nerve, is damaged by the high pressure. The damaged nerve cells can no longer carry information from the eye to the brain. Glaucoma is the most common cause of irreversible blindness in people over age 55. It is the leading cause of blindness in the United States, and is the leading cause of blindness in people over age 65 worldwide. People with glaucoma usually do not have any warning signs of the disease. Glaucoma is the leading cause of irreversible blindness in the United States. More than 600,000 Americans have glaucoma. If you have glaucoma, you may have noticed a sudden loss of vision, dimming of vision, blurred vision, halos around lights
